In [ ]:
import zipfile
import gzip
import json
from concurrent.futures import ThreadPoolExecutor

zip_filename = "chunk1.zip"

def process_json_data(compressed_data):
    with gzip.decompress(compressed_data) as decompressed_data:
        decoded_data = decompressed_data.decode('utf-8')
        json_data = json.loads(decoded_data)
        return [item for item in json_data['items'] if item.get('type') == 'peer-review']

with zipfile.ZipFile(zip_filename, 'r') as zip_file:
    with ThreadPoolExecutor() as executor:
        results = []
        for file_info in zip_file.infolist():
            if file_info.filename.startswith("chunk1/") and file_info.filename.endswith(".json.gz"):
                with zip_file.open(file_info) as compressed_file:
                    compressed_data = compressed_file.read()
                    results.append(executor.submit(process_json_data, compressed_data))
        
        peer_review_items = []
        for result in results:
            peer_review_items.extend(result.result())

output_filename = "peer_review_items.json"
with open(output_filename, 'w') as output_file:
    json.dump(peer_review_items, output_file, indent=4)

print("Peer-reviewed items saved to", output_filename)
